In [1]:
import json
from collections import defaultdict
import re


In [2]:

# Load your data
github_data = [
    {
      "pull_request": {
        "title": "Add automated tests for auth service",
        "author": "Priya",
        "description": "Added 10+ test cases to increase coverage by 22%",
        "reviewers": [
          {
            "reviewer": "Alex",
            "comment": "This is fantastic coverage. Really nice job!",
            "sentiment": "positive"
          },
          {
            "reviewer": "Jon",
            "comment": "Left a few minor comments, but overall LGTM.",
            "sentiment": "neutral"
          }
        ],
        "merged_by": "Mark",
        "acknowledged_contributor": "none"
      }
    },
    {
      "pull_request": {
        "title": "Fix flaky dashboard rendering bug",
        "author": "Jon",
        "description": "Resolved intermittent loading issue on dashboard page.",
        "reviewers": [
          {
            "reviewer": "Priya",
            "comment": "Thanks for fixing this — I hit this bug twice last week!",
            "sentiment": "positive"
          }
        ],
        "merged_by": "Jon",
        "acknowledged_contributor": "none"
      }
    },
    {
      "pull_request": {
        "title": "Update analytics pipeline for new metrics",
        "author": "Rina",
        "description": "Ingests new usage events and feeds them into the existing funnel structure.",
        "reviewers": [
          {
            "reviewer": "Alex",
            "comment": "Big change — I left a few questions but overall looks good.",
            "sentiment": "neutral"
          }
        ],
        "merged_by": "Rina",
        "acknowledged_contributor": "Priya"
      }
    }
  ]



transcript = [
    { "speaker": "Rina", "timestamp": "00:00:01", "text": "Morning, team! Quick sync today – mostly updates." },
    { "speaker": "Mark", "timestamp": "00:00:05", "text": "We’ve been blocked on the API test cases again." },
    { "speaker": "Alex", "timestamp": "00:00:09", "text": "I’ll jump on that after this, Mark." },
    { "speaker": "Rina", "timestamp": "00:00:12", "text": "Thanks Alex. Also, shoutout to Jon who stayed late to fix the CI/CD pipeline last night." },
    { "speaker": "Jon", "timestamp": "00:00:16", "text": "No problem — just didn’t want the deploy to slip." },
    { "speaker": "Mark", "timestamp": "00:00:20", "text": "Thanks, Jon. Also, Priya's async docs helped me a lot yesterday." },
    { "speaker": "Priya", "timestamp": "00:00:24", "text": "Glad to hear that! Let me know if it needs updates." },
    { "speaker": "Alex", "timestamp": "00:00:28", "text": "One note: design team has concerns about accessibility in the new UI. Anyone want to partner on that?" },
    { "speaker": "Rina", "timestamp": "00:00:33", "text": "Priya, you worked on this earlier, right?" },
    { "speaker": "Priya", "timestamp": "00:00:36", "text": "Yeah, I can draft the checklist again and work with design." },
    { "speaker": "Jon", "timestamp": "00:00:40", "text": "Awesome. I’ll document all this in our release notes." }
  ]



teams_chat = [
    {
      "sender": "Mark",
      "timestamp": "2025-04-08T13:20:00Z",
      "text": "Anyone have experience with the new Azure deployment config? Seeing weird errors."
    },
    {
      "sender": "Priya",
      "timestamp": "2025-04-08T13:21:30Z",
      "text": "Yeah, I ran into this last week. Try setting `enableDefaultIngress` to true."
    },
    {
      "sender": "Mark",
      "timestamp": "2025-04-08T13:23:10Z",
      "text": "That did the trick. Thanks!"
    },
    {
      "sender": "Jon",
      "timestamp": "2025-04-09T09:15:00Z",
      "text": "Need someone to sanity-check this SQL migration before I run it."
    },
    {
      "sender": "Alex",
      "timestamp": "2025-04-09T09:17:00Z",
      "text": "On it. Looking now."
    },
    {
      "sender": "Alex",
      "timestamp": "2025-04-09T09:22:00Z",
      "text": "Looks clean, just double-check the rollback logic."
    },
    {
      "sender": "Jon",
      "timestamp": "2025-04-09T09:23:00Z",
      "text": "Will do. Appreciate the quick help!"
    },
    {
      "sender": "Rina",
      "timestamp": "2025-04-10T11:00:00Z",
      "text": "Huge thanks to Priya for unblocking the deployment flow. Quietly crushed it again 👏"
    }
  ]



In [3]:

# Initialize scores
impact_scores = defaultdict(lambda: {
    "helped_others": 0,
    "received_public_praise": 0,
    "unacknowledged_contribs": 0,
    "proactive_support": 0,
    "total_score": 0
})

# ---------- GITHUB DATA ANALYSIS ----------
for pr in github_data:
    pr_data = pr["pull_request"]
    author = pr_data["author"]
    reviewers = pr_data["reviewers"]
    acknowledged = pr_data.get("acknowledged_contributor")

    # PR authorship is a base contribution — not silent unless ignored
    # If acknowledged contributor is present and not author, give score
    if acknowledged and acknowledged != author:
        impact_scores[acknowledged]["unacknowledged_contribs"] += 1

    for review in reviewers:
        sentiment = review["sentiment"]
        reviewer = review["reviewer"]
        if sentiment == "positive":
            impact_scores[reviewer]["helped_others"] += 1
        elif sentiment == "neutral":
            impact_scores[reviewer]["helped_others"] += 0.5

# ---------- TRANSCRIPT ANALYSIS ----------
for entry in transcript:
    text = entry["text"]
    speaker = entry["speaker"]

    # Look for praise or mentions by others
    for person in ["Priya", "Jon", "Rina", "Alex", "Mark"]:
        if person != speaker and re.search(rf"\b{person}\b", text, re.IGNORECASE):
            if any(word in text.lower() for word in ["thanks", "shoutout", "helped", "fix", "great job", "nice work", "awesome"]):
                impact_scores[person]["received_public_praise"] += 1

# ---------- TEAMS CHAT ANALYSIS ----------
for message in teams_chat:
    sender = message["sender"]
    text = message["text"]

    for person in ["Priya", "Jon", "Rina", "Alex", "Mark"]:
        if person != sender and re.search(rf"\b{person}\b", text, re.IGNORECASE):
            if "thanks" in text.lower() or "appreciate" in text.lower():
                impact_scores[person]["received_public_praise"] += 1

    # Look for helpful replies (technical advice, support)
    if any(word in text.lower() for word in ["try", "suggest", "set", "fix", "looking", "checked", "reviewed"]):
        impact_scores[sender]["proactive_support"] += 1

# ---------- FINAL SCORING ----------
# We can weight each category — tweak as needed
for person, metrics in impact_scores.items():
    score = (
        1.5 * metrics["helped_others"] +
        2.0 * metrics["unacknowledged_contribs"] +
        1.2 * metrics["received_public_praise"] +
        1.5 * metrics["proactive_support"]
    )
    metrics["total_score"] = round(score, 2)

# ---------- OUTPUT RESULTS ----------
sorted_impact = sorted(impact_scores.items(), key=lambda x: x[1]['total_score'], reverse=True)

print("Silent Impact Rankings:\n")
for person, metrics in sorted_impact:
    print(f"{person}: {metrics}")


Silent Impact Rankings:

Priya: {'helped_others': 1, 'received_public_praise': 2, 'unacknowledged_contribs': 1, 'proactive_support': 1, 'total_score': 7.4}
Alex: {'helped_others': 1.5, 'received_public_praise': 1, 'unacknowledged_contribs': 0, 'proactive_support': 1, 'total_score': 4.95}
none: {'helped_others': 0, 'received_public_praise': 0, 'unacknowledged_contribs': 2, 'proactive_support': 0, 'total_score': 4.0}
Jon: {'helped_others': 0.5, 'received_public_praise': 2, 'unacknowledged_contribs': 0, 'proactive_support': 0, 'total_score': 3.15}
